In [1]:
import os
import pathlib
import time

import findspark

prj_dir = pathlib.Path().resolve().parent.parent
spark_home = os.path.join(prj_dir / 'spark-3.5.0-bin-hadoop3')

findspark.init(spark_home)

from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext


conf = SparkConf().setMaster('local[2]').setAppName('Join Operation').set('spark.executer.memory', '2g')
sc = SparkContext(conf=conf)
ssc = StreamingContext(sparkContext=sc, batchDuration=10)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 15:08:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/audioworkstation/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/spark-3.5.0-bin-hadoop3/python/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [2]:
transaction_rdd_queue = []
for i in range(5): 
    transactions = [(customer_id, None) for customer_id in range(10)]
    transaction_rdd = ssc.sparkContext.parallelize(transactions)
    transaction_rdd_queue.append(transaction_rdd)

In [3]:
# Batch RDD of whether customers are good or bad. 
# (customer_id, is_good_customer)
customers = [(0,True),(1,False), (2,True), (3,False), (4,True), (5,False), (6,True), (7,False), (8,True), (9,False)]
customer_rdd = ssc.sparkContext.parallelize(customers)

In [4]:
# Creating queue stream
ds = ssc.queueStream(transaction_rdd_queue)

In [5]:
# Join the streaming RDD and batch RDDs to filter out bad customers.
dst = ds.transform(lambda rdd: rdd.join(customer_rdd)).filter(lambda rdd: rdd[1][1] == False)

## END OF EXERCISE SECTION ==================================
dst.pprint()

In [6]:
ssc.start()
# time.sleep(6)
# ssc.stop()

-------------------------------------------
Time: 2023-11-20 15:08:10
-------------------------------------------
(1, (None, False))
(5, (None, False))
(9, (None, False))
(3, (None, False))
(7, (None, False))

-------------------------------------------
Time: 2023-11-20 15:08:20
-------------------------------------------
(1, (None, False))
(5, (None, False))
(9, (None, False))
(3, (None, False))
(7, (None, False))

